# Fetch Archived Version of Fatal Encounter Supporting Links

In [32]:
!pip install wayback

Install libraries

In [35]:
import re
import time
import wayback
import pandas as pd

Pull table from Google Sheets

In [36]:
sheet_id = "1dKmaV_JiWcG8XBoRgP8b4e9Eopkpgt7FL7nyspvzAsE"
sheet_name = "sample_1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df_fe = pd.read_csv(url)
df_fe

,Unique ID,Name,Age,Gender,Race,Race with imputations,Imputation probability,URL of image (PLS NO HOTLINKS),Date of injury resulting in death (month/day/year),Location of injury (address),...,URL Temp,Brief description,"Dispositions/Exclusions INTERNAL USE, NOT FOR ANALYSIS",Intended use of force (Developing),Supporting document link,"Foreknowledge of mental illness? INTERNAL USE, NOT FOR ANALYSIS",Unnamed: 32,Unnamed: 33,Unique ID formula,Unique identifier (redundant)
0,31495.0,Ashley McClendon,28.0,Female,African-American/Black,African-American/Black,Not imputed,https://fatalencounters.org/wp-content/uploads...,12/31/2021,South Pearl Street and Tory Road,...,NaN,"Ashley McClendon's boyfriend, 33-year-old Marc...",Criminal,Pursuit,https://www.wsoctv.com/news/1-person-dead-afte...,No,NaN,NaN,NaN,31495.0
1,31496.0,Name withheld by police,NaN,Female,Race unspecified,NaN,NaN,NaN,12/31/2021,1500 21st Street,...,NaN,Police responded to a man causing a disturbanc...,Pending investigation,Deadly force,https://www.wtok.com/2022/01/01/officer-involv...,No,NaN,NaN,NaN,31496.0
2,31497.0,Name withheld by police,NaN,Male,Race unspecified,NaN,NaN,NaN,12/31/2021,1500 21st Street,...,NaN,Police responded to a man causing a disturbanc...,Pending investigation,Deadly force,https://www.wtok.com/2022/01/01/officer-involv...,No,NaN,NaN,NaN,31497.0
3,31491.0,Johnny C. Martin Jr.,36.0,Male,Race unspecified,NaN,NaN,NaN,12/30/2021,Martinez Lane,...,NaN,"Johnny C. Martin, Jr. arrived at a gas station...",Suicide,Suicide,https://gbi.georgia.gov/press-releases/2021-12...,No,NaN,NaN,NaN,31491.0
4,31492.0,Dennis McHugh,44.0,Male,European-American/White,NaN,NaN,NaN,12/30/2021,435 E 4th Street,...,NaN,Deputies responded to a domestic violence call...,Pending investigation,Deadly force,https://kesq.com/news/2021/12/31/officer-invol...,No,NaN,NaN,NaN,31492.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31493,25746.0,Samuel H. Knapp,NaN,Male,European-American/White,European-American/White,Not imputed,NaN,01/01/2000,27898-27804 US-101,...,NaN,Samuel Knapp was allegedly driving a stolen ve...,Unreported,Pursuit,https://drive.google.com/file/d/10DisrV8K5ReP1...,No,NaN,NaN,NaN,25746.0
31494,25747.0,Mark A. Horton,NaN,Male,African-American/Black,African-American/Black,Not imputed,NaN,01/01/2000,Davison Freeway,...,NaN,Two Detroit men killed when their car crashed ...,Unreported,Pursuit,https://drive.google.com/file/d/1-nK-RohgiM-tZ...,No,NaN,NaN,NaN,25747.0
31495,25748.0,Phillip A. Blurbridge,NaN,Male,African-American/Black,African-American/Black,Not imputed,NaN,01/01/2000,Davison Freeway,...,NaN,Two Detroit men killed when their car crashed ...,Unreported,Pursuit,https://drive.google.com/file/d/1-nK-RohgiM-tZ...,No,NaN,NaN,NaN,25748.0
31496,25749.0,Mark Ortiz,NaN,Male,Hispanic/Latino,Hispanic/Latino,Not imputed,NaN,01/01/2000,600 W Cherry Ln,...,NaN,A motorcycle was allegedly being driven errati...,Unreported,Pursuit,https://drive.google.com/file/d/1qAEefRjX_aTtC...,No,NaN,NaN,NaN,25749.0


Filter for the records we have in Wikidata (African Americans from California)

In [37]:
ca_only_df = df_fe.loc[df_fe['State'] == 'CA']
ca_only_df_2 = ca_only_df.loc[ca_only_df['Race'] == 'African-American/Black']
len(ca_only_df_2)

723

Function for calling getting links from wayback lib

In [38]:
session = wayback.WaybackSession(retries=25)
wb = wayback.WaybackClient(session)

def archived_page(url):
    time.sleep(.5)
    if pd.isna(url):  # especially for MEI column that has lots of missing links
        return '--No link for this in original dataset--'
    try:
        for result in wb.search(url):
            if result.status_code == 200:
                return result.view_url
    except wayback.exceptions.BlockedSiteError:
        return None

# archived_page(urls[0])

Apply archived_page function to the 'Supporting document link' column

In [41]:
archived_supporting_urls = ca_only_df_2['Supporting document link'].apply(archived_page)

In [46]:
ca_fe_urls_2 = pd.DataFrame({"Unique ID": ca_only_df_2['Unique ID'], "Name": ca_only_df_2['Name'], "archived supporting link": archived_supporting_urls})
ca_fe_urls_2

,Unique ID,Name,archived supporting link
84,31331.0,Amanuel Aman Moreno,https://web.archive.org/web/20211213144129/htt...
171,31266.0,Devin Edward Hall,https://web.archive.org/web/20220529020741/htt...
582,30882.0,Cedric Baxter,https://web.archive.org/web/20210904011902/htt...
621,30976.0,Amelian Tremond Houff,https://web.archive.org/web/20210925225407/htt...
642,30831.0,Robert Anderson,https://web.archive.org/web/20210827045055/htt...
...,...,...,...
31213,190.0,Randall Raymond Ramsey,https://web.archive.org/web/20160517182234/htt...
31254,156.0,Thomas Allen Freeman Jr.,https://web.archive.org/web/20201108070237/htt...
31405,57.0,Sonserra Holloway,https://web.archive.org/web/20110129021654/htt...
31428,38.0,Charles Hill,None


In [47]:
ca_fe_urls_1.to_csv('ca_wiki_fe_archived_links.csv', encoding='utf-8')

Code that kept getting me disconnected:

In [30]:
archived_supporting_urls = []
for url in ca_only_df_2['Supporting document link']:
    ar_url = archived_page(url)
    archived_supporting_urls.append(ar_url)
    print(ar_url)

https://web.archive.org/web/20211213144129/https://www.nbcbayarea.com/news/local/off-duty-uc-berkeley-officer-shoots-kills-armed-robber-at-san-pablo-restaurant/2754237/
https://web.archive.org/web/20220529020741/https://mynewsla.com/crime/2021/11/26/gunman-fatally-shot-after-allegedly-killing-man-in-pasadena-3/
https://web.archive.org/web/20210904011902/https://www.nbclosangeles.com/news/local/police-fatally-shoot-pursuit-suspect-who-crashed-into-buena-park-school-district-parking-lot/2686134/
https://web.archive.org/web/20210925225407/https://www.abc10.com/article/news/community/race-and-culture/mother-seeking-answers-after-son-dies-in-police-chase-in-sacramento/103-8223d5d4-34d3-4847-a316-728c0c9ab632
https://web.archive.org/web/20210827045055/https://www.triplicate.com/news_free/black-man-killed-in-officer-involved-shooting/article_43fccdb4-06da-11ec-84ea-37cde00b43ec.html
https://web.archive.org/web/20210720004444/https://ktla.com/news/local-news/motorcyclist-suspected-of-street-ra

https://web.archive.org/web/20191101201021/https://www.nytimes.com/2019/11/01/us/echo-park-crash.html
https://web.archive.org/web/20191107151235/https://www.kcra.com/article/video-deadly-carmichael-shootings/29714085
https://web.archive.org/web/20191107151235/https://www.kcra.com/article/video-deadly-carmichael-shootings/29714085
https://web.archive.org/web/20230725163319/https://homicide.latimes.com/post/delfon-garnell-kinney-sr/
https://web.archive.org/web/20191211131556/https://www.canyon-news.com/murder-suspect-killed-in-officer-involved-shooting/101979
https://web.archive.org/web/20210228225325/https://homicide.latimes.com/post/sawandi-toussaint/
https://web.archive.org/web/20191014085401/https://www.abc10.com/article/news/crime/sacramento-sheriffs-deputy-involved-shooting/103-494bde63-041f-40fc-902a-df64bae84080
https://web.archive.org/web/20190924053719/https://abc7.com/suspect-fatally-shot-by-police-in-long-beach-was-holding-fake-gun-police-say-/5560480/
https://web.archive.org

https://web.archive.org/web/20171115085541/http://www.recordnet.com/news/20171114/stockton-police-shoot-kill-knife-wielding-man
https://web.archive.org/web/20230725163040/https://www.kerncounty.com/home/showdocument?id=3854
https://web.archive.org/web/20190725153300/https://www.pe.com/2019/07/24/details-from-2017-castle-view-elementary-standoff-released-for-first-time-in-das-investigatory-report/
https://web.archive.org/web/20170929194634/http://www.sfgate.com/crime/article/Man-dies-after-Oakland-use-stun-gun-to-detain-him-12240565.php
https://web.archive.org/web/20170928163442/http://www.sfgate.com/crime/article/Homicide-suspect-ID-d-in-I-80-officer-involved-12237761.php
https://web.archive.org/web/20200207213051/https://sanfrancisco.cbslocal.com/2019/08/13/sfpd-officer-involved-shootings-oliver-barcenas-damien-murray-no-charges/
https://web.archive.org/web/20181116080038/https://www.northcoastjournal.com/NewsBlog/archives/2018/11/13/officers-cleared-in-fatal-plaza-shooting-with-video

https://web.archive.org/web/20160210044017/http://abc7news.com/news/3-killed-after-police-chase-fiery-crash-in-san-francisco-identified/1192752/
https://web.archive.org/web/20161007193303/http://www.eastbaytimes.com/2016/10/06/antioch-man-put-in-police-neck-hold-died-of-asphyxiation-jury-rules-it-an-accident/
https://web.archive.org/web/20160101124220/http://abc7.com/news/devoted-church-man-killed-in-fiery-hollywood-car-crash/1137885/
https://web.archive.org/web/20160221210456/http://www.nbclosangeles.com/news/local/Overnight-Deputy-Involved-Shooting-in-Palmdale-363065491.html
https://web.archive.org/web/20151227211350/http://www.contracostatimes.com/breaking-news/ci_29300636/hayward-man-dies-while-police-custody
https://web.archive.org/web/20151224162527/http://www.contracostatimes.com/breaking-news/ci_29263183/antioch-man-faces-murder-charges-crash-that-killed?source=pkg
https://web.archive.org/web/20151215002536/http://abc7.com/news/suspect-killed-in-lynwood-deputy-involved-shooting

https://web.archive.org/web/20140811200728/http://sfappeal.com/2014/06/woman-dies-in-crash-after-allegedly-fleeing-from-chp/
https://web.archive.org/web/20161201214214/http://www.usatoday.com/pages/interactives/blacks-killed-police-chases-higher-rate/
https://web.archive.org/web/20201229010842/https://abc7.com/universal-citywalk-shooting-officer-involved-james-white/55722/
https://web.archive.org/web/20230725164330/https://www.pressreader.com/usa/imperial-valley-press/20180215/281492161785799
None
https://web.archive.org/web/20160708135523/http://www.modbee.com/news/local/crime/article88027087.html
https://web.archive.org/web/20201023055720/https://www.kcra.com/article/police-had-stopped-pursuit-before-crash-killing-3-teens/6413257
https://web.archive.org/web/20201023055720/https://www.kcra.com/article/police-had-stopped-pursuit-before-crash-killing-3-teens/6413257
https://web.archive.org/web/20140515001935/http://www.latimes.com/local/lanow/la-me-ln-suspect-in-lapd-station-shooting-20

https://web.archive.org/web/20120607020609/http://www.nbcbayarea.com/news/local/Police-Shoot-Kill-South-SF-Teen-157645885.html
https://web.archive.org/web/20151006043217/http://www.latimes.com/local/crime/la-me-lapd-chokehold-jail-20151006-story.html
https://web.archive.org/web/20160305232717/http://www.mercedsunstar.com/news/local/article3267742.html
https://web.archive.org/web/20120701140218/http://www.sfgate.com/crime/article/Fatal-shooting-by-Vallejo-police-is-2nd-in-3-days-3590418.php
https://web.archive.org/web/20120525073541/http://www.nbcsandiego.com/news/local/Man-Shot-Killed-Oceanside-Shooting-Identified-152598405.html
https://web.archive.org/web/20121015012447/http://oaklandnorth.net/2012/10/09/district-attorneys-report-officer-justified-in-alan-blueford-shooting/
None
https://web.archive.org/web/20160718123406/http://www.nbclosangeles.com/news/local/Porn-Actor-Sledge-Hammer-Marland-Anderson-Dies-Encounter-LAPD-Paramedics-Reseda-147646705.html
None
https://web.archive.org/we

https://web.archive.org/web/20150801032811/http://www.laweekly.com/news/alleged-porn-star-killer-dies-after-police-shoot-and-he-falls-off-a-cliff-2390630
https://web.archive.org/web/20170823192920/http://homicide.latimes.com/post/donovan-morris/
https://web.archive.org/web/20130526165843/http://blogs.lawyers.com/2013/05/police-shooting-victims-family-wins-8m/
https://web.archive.org/web/20140717103022/http://homicide.latimes.com/post/steven-washington/
https://web.archive.org/web/20150801011717/https://positiveleo.wordpress.com/2010/09/17/hero-cops-dumped-oakland-for-hayward/
https://web.archive.org/web/20200927172645/https://www.vcdistrictattorney.com/wp-content/uploads/2013/02/ois_report_final_123010.pdf
None
https://web.archive.org/web/20140429084239/http://www.sfgate.com/bayarea/article/Oakland-police-kill-one-suspect-wound-another-3208361.php
None
None
https://web.archive.org/web/20161114083011/http://homicide.latimes.com/post/travion-richard/
https://web.archive.org/web/201611111

https://web.archive.org/web/20150523123534/http://www.sbsun.com/general-news/20120125/lawsuit-settled-over-sb-police-shooting
None
https://web.archive.org/web/20161111184345/http://homicide.latimes.com/post/jamar-witherspoon/
https://web.archive.org/web/20160509152328/http://homicide.latimes.com/post/ronald-ball/
None
https://web.archive.org/web/20150302082929/http://homicide.latimes.com/post/richard-tyson/
https://web.archive.org/web/20150523123534/http://www.sbsun.com/general-news/20120125/lawsuit-settled-over-sb-police-shooting
None
https://web.archive.org/web/20141216220921/http://www.pressdemocrat.com/news/2258313-181/chass-lawsuit-settled-for-175
https://web.archive.org/web/20150629024606/http://www.sfgate.com/bayarea/article/SAN-FRANCISCO-Police-shoot-kill-man-wielding-2611721.php
https://web.archive.org/web/20071011072338/http://www.lapdonline.org/march_2007/news_view/34896
https://web.archive.org/web/20071011071030/http://www.lapdonline.org/february_2007/news_view/34825
None
h

https://web.archive.org/web/20041221035413/http://www.mindfully.org/Technology/2004/Taser-Victim-Dies17sep04.htm
https://web.archive.org/web/20201101011252/https://www.sfgate.com/bayarea/article/VALLEJO-Police-identify-shooting-victims-2-2694321.php
https://web.archive.org/web/20120128171622/http://www.ecdlaw.info/outlines/4-2-08%20Sanders%20(CA)%20Order%20Granting%20City%20Def's%20MSJ.pdf
None
https://web.archive.org/web/20221228181507/https://homicide.latimes.com/post/keyante-joseph-reed/
https://web.archive.org/web/20230725163829/https://homicide.latimes.com/post/lavelle-rigmaiden/
https://web.archive.org/web/20230725163125/https://www.sfgate.com/bayarea/article/BAY-POINT-Coroner-rules-death-of-man-an-accident-2698258.php
https://web.archive.org/web/20160516183850/http://www.sfgate.com/bayarea/article/SAN-FRANCISCO-Police-kill-suspect-in-street-2710737.php
https://web.archive.org/web/20200904025231/https://www.sfgate.com/bayarea/article/OAKLAND-Attempted-murder-suspect-shot-killed-2

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

## All of California

Apply archived_page function to all records from California

In [48]:
archived_all_ca = ca_only_df['Supporting document link'].apply(archived_page)

WaybackException: 400 Client Error: Bad Request for url: https://web.archive.org/cdx/search/cdx?url=http%3A%2F%2Fmhttps%3A%2F%2Fmadriverunion.com%2Fmotorcyclist-dead-after-vehicle-pursuit%2Fadriverunion.com%2Fmotorcyclist-dead-after-car-pursuit%2F&limit=1000&showResumeKey=true&resolveRevisits=true

Make DF from these links

In [ ]:
ca_df_all_archive = pd.DataFrame({'Unique ID': ca_only_df['Unique ID'], 'Name': ca_only_df['Name'], 'archived supporting link': archived_all_ca})

Export into CSV

In [ ]:
ca_df_all_archive.to_csv('ca_ALL_fe_ar_links.csv', encoding='utf-8')

In [24]:
archived_supporting_urls = []
for url in ca_only_df['Supporting document link'][1853:]:
    ar_url = archived_page(url)
    archived_supporting_urls.append(ar_url)
    print(ar_url)

None
https://web.archive.org/web/20150322072944/http://www.sbsun.com/general-news/20140312/highland-deputies-shoot-kill-hatchet-wielding-man
None
https://web.archive.org/web/20140312174413/http://collegian.csufresno.edu/2014/03/07/woman-shot-twice-killed-by-fresno-police-officer-in-apartment-complex-west-of-campus/
https://web.archive.org/web/20140511163945/http://homicide.latimes.com/post/kenny-clinton-walker/
None
https://web.archive.org/web/20141025183008/http://www.sacbee.com/news/local/crime/article2592271.html
https://web.archive.org/web/20150326171805/http://www.utsandiego.com/news/2014/feb/26/suicidal-man-rifle-downtown-san-diego/
https://web.archive.org/web/20140302162426/http://abclocal.go.com/kfsn/story?section=news/local&id=9442643
https://web.archive.org/web/20140221233207/http://sanfrancisco.cbslocal.com/2014/02/21/reports-of-officer-involved-shooting-near-san-jose-sate-university/
https://web.archive.org/web/20140311040954/http://www.nbcsandiego.com/news/local/Shooting-S

https://web.archive.org/web/20150123161631/http://homicide.latimes.com/post/paul-aguilar/
https://web.archive.org/web/20131114005253/http://ktla.com/2013/11/13/hawthorne-police-fatally-shoot-man/
https://web.archive.org/web/20150801001856/http://www.sbsun.com/general-news/20121114/investigators-man-shot-killed-by-sheriffs-deputies-threatened-to-kill-firefighters-deputies
https://web.archive.org/web/20161216031146/http://da.lacounty.gov/sites/default/files/pdf/JSID_OIS_Jordan.pdf
None
https://web.archive.org/web/20200708172417/https://www.dailybulletin.com/2013/11/07/upland-man-idd-in-explosive-riverside-crash/
https://web.archive.org/web/20200708172417/https://www.dailybulletin.com/2013/11/07/upland-man-idd-in-explosive-riverside-crash/
https://web.archive.org/web/20210507143215/https://abc30.com/archive/9313809/
https://web.archive.org/web/20150801024942/http://www.kcra.com/news/officerinvolved-shooting-shuts-down-hwy-12-near-fairfield/22812924
https://web.archive.org/web/201407220119

https://web.archive.org/web/20201125014741/https://thesheetnews.com/2013/08/07/mono-county-coroner-investigation-into-suicide-of-coleville-resident/
None
None
https://web.archive.org/web/20190228131007/http://orangecountyda.org/civicax/filebank/blobdload.aspx?BlobID=22806
https://web.archive.org/web/20190228130754/http://orangecountyda.org/civicax/filebank/blobdload.aspx?BlobID=22820
None
https://web.archive.org/web/20200609112847/http://www.montereycountyweekly.com/blogs/news_blog/article_696e4272-f8af-11e2-b8f0-001a4bcf6878.html
None
https://web.archive.org/web/20181116112314/https://www.psblaw.com/2014/12/11/13-million-settlement-for-family-of-woman-killed-by-hawthorne-police-vehicle/
https://web.archive.org/web/20161029100852/http://www.whittierdailynews.com/general-news/20130727/mother-questions-fatal-west-whittier-deputy-involved-shooting
https://web.archive.org/web/20230725163830/https://patch.com/california/suisuncity/update-man-shot-friday-by-pd-identified-as-suisun-resident-l

https://web.archive.org/web/20130621134933/http://sanfrancisco.cbslocal.com/2013/03/25/man-shot-by-deputies-in-castro-valley-dies-in-hospital/
https://web.archive.org/web/20130325073302/http://latimesblogs.latimes.com/lanow/2013/03/man-with-replica-pistol-killed-by-huntington-beach-police.html
https://web.archive.org/web/20131215072441/http://fox40.com/2013/03/26/elderly-modesto-man-killed-in-officer-involved-shooting-identified/
None
None
https://web.archive.org/web/20141108113131/http://www.sccgov.org/sites/da/newsroom/newsreleases/Documents/Elias-Mejia_OIS_04232014.pdf
None
https://web.archive.org/web/20131205035530/http://www.dailybulletin.com/general-news/20130319/officer-involved-shooting-in-pomona-still-under-investigation
https://web.archive.org/web/20130323040933/http://www.fugitive.com/2013/03/19/angelo-d-moreno-shot-by-american-canyon-police-officers/
https://web.archive.org/web/20170216002300/http://orangecountyda.org/civicax/filebank/blobdload.aspx?BlobID=23348
None
https:

https://web.archive.org/web/20121128050930/http://www.nbcsandiego.com/news/local/Murder-Suspect-Dies-After-Standoff-180608881.html
None
None
None
https://web.archive.org/web/20150801024906/http://www.scpr.org/blogs/news/2012/11/16/11129/paramount-man-killed-deputy-involved-shooting/
https://web.archive.org/web/20200804180121/https://www.visaliatimesdelta.com/story/news/2014/04/04/update-teen-armed-in-tulare-officer-involved-shooting/7312603/
https://web.archive.org/web/20121117190600/http://www.theavtimes.com/2012/11/14/man-dies-in-crash-after-high-speed-pursuit/
https://web.archive.org/web/20150731234949/http://www.desertsun.com/story/news/crime_courts/2014/10/30/cpl-allan-devillena-ii-police-shooting-chad-nordman-mike-heron/18194959/
https://web.archive.org/web/20130130214519/http://raniakhalek.com/2013/01/28/la-sheriffs-deputies-shot-unarmed-latino-father-of-2-in-the-back-7-times/
None
None
https://web.archive.org/web/20121103205606/http://www.nbcsandiego.com/news/local/Officer-Invo

https://web.archive.org/web/20150801023318/http://www.cbs8.com/story/19039202/woman-wounded-in-police-shooting-dies
None
None
https://web.archive.org/web/20150801011419/http://www.kcra.com/news/local-news/news-modesto/Stanislaus-deputy-fatally-shot-man-in-standoff/15405020
https://web.archive.org/web/20150801002956/http://abc7news.com/archive/8724548/
None
https://web.archive.org/web/20200813092208/http://www.crimevoice.com/2012/07/06/man-dead-after-officer-involved-shooting-in-orange-cove/
https://web.archive.org/web/20120713223343/http://www.lompocrecord.com/news/local/crime-and-courts/deputy-on-leave-after-fatal-shooting/article_c82000f8-c72d-11e1-ade8-001a4bcf887a.html
https://web.archive.org/web/20150801024834/http://www.pe.com/articles/deputy-650722-castillo-sheriff.html
None
https://web.archive.org/web/20130806025112/http://www.countyofsb.org/da/msm_county/documents/JoseLuisOrtegaReport.pdf
https://web.archive.org/web/20150731234225/http://www.mercurynews.com/breaking-news/ci_20

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [26]:
fe_urls_2 = pd.DataFrame({"Unique ID": df_fe['Unique ID'][18, "Name": df_fe['Name'], "archived supporting link": archived_supporting_urls})
fe_urls_2

ValueError: array length 542 does not match index length 31498

In [13]:
archived_supporting_urls = []
for url in df_fe['Supporting document link'][649:1300]:
    ar_url = archived_page(url)
    archived_supporting_urls.append(ar_url)
    print(ar_url)

https://web.archive.org/web/20210824215038/https://wblk.com/police-chase-in-buffalo-ends-with-one-dead-one-in-critical-condition/
https://web.archive.org/web/20210828145030/https://www.abc4.com/news/local-news/one-dead-after-officer-involved-shooting-in-provo/
https://web.archive.org/web/20210829192239/https://www.cbs46.com/news/22-year-old-man-killed-in-officer-involved-shooting-no-weapon-found-at-the-scene/article_662f2790-05ba-11ec-a3cc-bbaa1727e90f.html
https://web.archive.org/web/20210823193922/https://www.wsls.com/news/2021/08/23/southbound-i-81-in-montgomery-county-shut-down/
https://web.archive.org/web/20210824035004/https://www.amarillo.com/story/news/2021/08/23/amarillo-police-investigating-2nd-officer-involved-shooting/8252561002/
https://web.archive.org/web/20210823211717/https://sacramento.cbslocal.com/2021/08/23/galt-officers-hwy-99-head-on-crash/
https://web.archive.org/web/20210822141950/https://www.jsonline.com/story/news/crime/2021/08/22/greenfield-police-officer-shot

https://web.archive.org/web/20210812060303/https://www.kitv.com/story/44503410/investigation-continues-after-policeinvolved-shooting-in-kalihi-suspect-dead
https://web.archive.org/web/20210811125140/https://www.tampabay.com/news/clearwater/2021/08/11/woman-killed-after-driver-flees-traffic-stop-crashes-into-building-in-clearwater-police-say/
https://web.archive.org/web/20210811194135/https://signalscv.com/2021/08/suspect-down-in-deputy-involved-shooting-near-templin-highway/
None
https://web.archive.org/web/20210811211108/https://www.walb.com/2021/08/11/sheriff-chase-ends-fatal-wreck-i-75/
https://web.archive.org/web/20210812001018/https://kdvr.com/news/local/adams-county-sheriffs-office-responds-to-officer-involved-shooting-no-threat-to-public/
https://web.archive.org/web/20210811205922/https://kmph.com/news/local/swat-has-been-called-to-the-flamingo-mobile-home-lodge
None
https://web.archive.org/web/20210810222513/https://ktar.com/story/4623774/el-mirage-police-fatally-shoot-suspect-

None
https://web.archive.org/web/20210928212321/https://www.ajc.com/news/3-dead-1-injured-in-3-cobb-crashes/5AZ5G7LE4RAUNESNTTRVYTFV4A/
https://web.archive.org/web/20210818031927/https://www.twincities.com/2021/08/17/wisconsin-man-dies-two-weeks-after-car-crash-during-police-chase-officials-say/
https://web.archive.org/web/20210809082247/https://www.njoag.gov/updated-attorney-generals-office-investigating-fatal-accident-in-which-police-vehicle-struck-pedestrian-in-east-brunswick/
None
None
https://web.archive.org/web/20210731201420/https://www.news4jax.com/news/local/2021/07/31/officer-involved-shooting-reported-in-yulee/
https://web.archive.org/web/20210801055049/https://www.foxnews.com/us/arizona-border-patrol-agent-another-driver-killed-in-head-on-crash
None
https://web.archive.org/web/20210803225357/https://www.yourcentralvalley.com/news/crime/woodville-murder-suspect-dies-after-he-was-shot-by-deputies-sheriffs-office-says/
https://web.archive.org/web/20210731200324/https://www.fox

KeyboardInterrupt: 

In [ ]:
fe_urls.to_csv('fe_archived_links.csv', encoding='utf-8')